In [20]:
ENV["GRB_LICENSE_FILE"] = "/Users/solvibjornsson/Gurobi/gurobi.lic"

"/Users/solvibjornsson/Gurobi/gurobi.lic"

In [21]:
using XLSX
using DataFrames

# Read data from Excel file
xf = XLSX.readxlsx("facilityData.xlsx")



function read_column(sheet, firstrow, col)
    res = []
    row = firstrow
    while !ismissing(sheet[row,col])
        push!(res,sheet[row,col])
        row+=1
    end 
    return res
end


function Customer_data(xf, sheet_name)
    sheet = xf[sheet_name]

    firstrow = 2
    col = 1
    customer = read_column(sheet, firstrow, col)

    col = 2
    latitude = read_column(sheet, firstrow, col)

    col = 3
    longitude = read_column(sheet, firstrow, col)

    col = 4
    visits = read_column(sheet, firstrow, col)

    rawdata = DataFrame(customer = customer, latitude = latitude, 
            longitude = longitude, visits = visits)
    return rawdata
end

function facilities_data(xf, sheet_name)
    sheet = xf[sheet_name]

    firstrow = 2
    col = 1
    facility = read_column(sheet, firstrow, col)

    col = 2
    latitude = read_column(sheet, firstrow, col)

    col = 3
    longitude = read_column(sheet, firstrow, col)

    col = 4
    closing_cost = read_column(sheet, firstrow, col)
    
    col = 5
    capacity = read_column(sheet, firstrow, col)

    col = 6
    type = read_column(sheet, firstrow, col)

    rawdata = DataFrame(facility = facility, latitude = latitude, 
            longitude = longitude, closing_cost = closing_cost, capacity = capacity, type = type)
    return rawdata
end


customers_data = Customer_data(xf, "Customers")

facility_data = facilities_data(xf, "Fac")


Row,facility,latitude,longitude,closing_cost,capacity,type
,Any,Any,Any,Any,Any,Any
1,Zurich West,47.3837,8.5265,20000,0,480
2,Geneva South,46.1793,6.3196,32500,0,288
3,Luzern North,47.0813,8.3017,15000,0,270
4,Bern North,46.9612,7.4358,0,75000,240
5,Lausanne,46.5152,6.6522,0,56250,300
6,Geneva North,46.3886,6.2428,0,68750,260
7,Altdorf,46.879,8.6483,0,62500,200
8,Montreux,46.4353,6.9167,0,37500,140
9,Zug,47.1525,8.5108,0,60000,160


# Routing Technicians within Z¨urich

In [22]:
#module Routing_Tech1
using JuMP
using Gurobi
# Call the function with your file's path and sheet name
using XLSX
using DataFrames

# Function to read Excel data into a matrix, skipping the first row and column
function read_excel_to_matrix(file_path, sheet_name)
    # Load the workbook
    wb = XLSX.readxlsx(file_path)

    # Get the specified sheet
    sheet = wb[sheet_name]

    # Read the data into an array, dropping the first row and column
    data = XLSX.getdata(sheet)#, drop=1:1, droprows=1)
    data=data[2:end,2:end]
    
    
    # Convert the array to a matrix
    #matrix_data = hcat(data...)
    return data
end

distances_matrix = read_excel_to_matrix("vrpData.xlsx", "distances")
#repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
distances_matrix




31×31 Matrix{Any}:
  0         4         9.13017  10        …   2.72029   7.65506  1.23693
  4         0         8.00999   7.2111       6.40312   4.12311  2.08087
  9.13017   8.00999   0         4.11825     11.6499   10.9636   5.53624
 10         7.2111    4.11825   0           12.7201    8.63713  5.59732
  5.38516   2.23607  10.018     8.544        7.2111    2.28035  2.20907
  5         8.06226  14.0982   14.8661   …   2.52982  10.2956   1.96977
  5.09902   7.07107  14.0769   14.2127       3.60555   8.544    1.52643
  6.32456  10.198    14.4278   16.1245       3.82099  13.2136   3.30606
 12.7279   15.8114   13.4373   17.2627      12.5936   19.9249   7.45185
  4.24264   7.61577   8.93085  11.4018       4.66905  11.6362   3.245
  ⋮                                      ⋱                      ⋮
 12.0416   13        20.7162   20.1246      10.469    12.474    4.86621
 12.2703   11.6      19.6041   18.0488      11.7661    9.55196  4.90408
 18.4391   22.3607   25.4118   27.8568      15.9812  

In [30]:
#module Routing_Tech1
using JuMP
using Gurobi
# Call the function with your file's path and sheet name
using XLSX
using DataFrames



# Function to read Excel data into a matrix, skipping the first row and column
function read_excel_to_matrix(file_path, sheet_name)
    # Load the workbook
    wb = XLSX.readxlsx(file_path)

    # Get the specified sheet
    sheet = wb[sheet_name]

    # Read the data into an array
    data = XLSX.getdata(sheet)

    # Convert the array to a matrix and remove the first row and column
    # The `[2:end, 2:end]` slicing skips the first row and the first column
    matrix_data = convert(Matrix, data)[2:end, 2:end]

    return matrix_data
end

# Replace "vrpData.xlsx" with the actual path to your Excel file
distances_matrix = read_excel_to_matrix("vrpData.xlsx", "distances")
#repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")

function vrp_load_constraint()

    #------
    # DATA
    #------

    K = 10 # number vehicles
    N = 31; # number of nodes, node 1 is the depot

    C = 6; #capa for working hours excluding driving

    d = repair_times

    c = distances_matrix
    
    #------
    # MODEL
    #------

    model = Model(Gurobi.Optimizer);

    @variable(model, x[1:N,1:N,1:K] >= 0, Bin); #node to node coverd by truck
    @variable(model, y[1:N,1:K] >= 0, Bin); #
    @variable(model, z[1:N,1:K] >= 0); # load of truck k when arriving at node i

    @objective(model, Min, sum( sum( sum( c[i,j]*x[i,j,k]  for j in 1:N) for i in 1:N) for k = 1:K)); #mimize distance covered
    

    @constraint(model,[i = 2:N], sum(y[i,k] for k in 1:K) == 1); #every cust visited
    @constraint(model,[h = 1:N, k = 1:K], sum(i == h ? 0 : x[i,h,k] for i = 1:N) == y[h,k]); #inflow = outflow
    @constraint(model,[h = 1:N, k = 1:K], sum(j == h ? 0 : x[h,j,k] for j = 1:N) == y[h,k]); #inflow = outflow
    @constraint(model,[k = 1:K], sum(d[i]*y[i,k] for i = 2:N) <= C); #not exceeding work hours capa

    @constraint(model,[i = 2:N, j = 2:N, k = 1:K], z[i,k] - d[i] >= z[j,k] - (1-x[i,j,k])*sum(d));  #load constraint - subtour elimi

    #-------
    # SOLVE
    #-------
    set_optimizer_attribute(model, "TimeLimit", 30);
    optimize!(model)
    

    println();
    for k = 1:K
        has_route = false;
        for i = 1:N
            for j = 1:N
                if (value(x[i,j,k]) == 1)
                    print(i-1,"->",j-1," ")
                    has_route = true;
                end
            end
        end
        if has_route
            println();
        end
    end

    print("\ntotal cost = ");
    println(objective_value(model));

end

vrp_load_constraint();




Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-20
Set parameter TimeLimit to value 60
Set parameter TimeLimit to value 60
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.6.0 22G436)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9660 rows, 10230 columns and 46220 nonzeros
Model fingerprint: 0x7e32c2db
Variable types: 310 continuous, 9920 integer (9920 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+01]
  Objective range  [6e-01, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 300 rows and 320 columns
Presolve time: 0.03s
Presolved: 9360 rows, 9910 columns, 45920 nonzeros
Variable types: 300 continuous, 9610 integer (9610 binary)

Root relaxation: objective 1.131945e+02, 990 iterations, 0.02 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Ex

10. Location-based heuristic

In [29]:
#module Routing_Tech1
using JuMP
using Gurobi
# Call the function with your file's path and sheet name
using XLSX
using DataFrames



# Function to read Excel data into a matrix, skipping the first row and column
function read_excel_to_matrix(file_path, sheet_name)
    # Load the workbook
    wb = XLSX.readxlsx(file_path)

    # Get the specified sheet
    sheet = wb[sheet_name]

    # Read the data into an array
    data = XLSX.getdata(sheet)

    # Convert the array to a matrix and remove the first row and column
    # The `[2:end, 2:end]` slicing skips the first row and the first column
    matrix_data = convert(Matrix, data)[2:end, 2:end]

    return matrix_data
end


# Replace "vrpData.xlsx" with the actual path to your Excel file

#repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")
c_ij = read_excel_to_matrix("vrpData.xlsx", "c_ij_new")
repair_times = read_excel_to_matrix("vrpData.xlsx", "repairTimes")

using JuMP
using Gurobi

function solve_facility_location()

    #------
    # DATA
    #------

    n = 30; 
    m = 30; # number of customers

    #f = Parameter, the cost of selectiong cust n as a cluster rep
    f = [8	18.26033954	20	10.77032961	10	10.19803903	12.64911064	25.45584412	8.485281374	17.11724277	21.28003759	18.02775638	4.472135955	24.75883681	12.16552506	13.41640786	21.9317122	25.61249695	39.59797975	34	22.91200559	24.08318916	24.54057864	36.87817783	22.36067977	27.91128804	9.666436779	5.440588203	15.31012737	2.473863375];
    # c: Parameter, the cost of assigning cust n to the cluster that cust m represents
    c = read_excel_to_matrix("vrpData.xlsx", "c_ij_new");
    d = read_excel_to_matrix("vrpData.xlsx", "repairTimes");
    d=d[2:end]; #exclude the depot

    C = 6; #we have γ = 1, so same capacity

    #------
    # MODEL
    #------

    model = Model(Gurobi.Optimizer);

    @variable(model, x[1:n] >= 0, Bin); #1 if cust n is chosen as "cluster representative"
    @variable(model, y[1:n,1:m] >= 0, Bin); #cust n is assigned to cluster that cust m represnts

    @objective(model, Min, sum( f[j]*x[j] for j in 1:n) + sum( sum( c[i,j]*y[i,j] for j in 1:n) for i = 1:m));

    @constraint(model,[i = 1:m], sum(y[i,j] for j in 1:n) == 1);
    @constraint(model,[j = 1:n], sum( d[i]*y[i,j] for i = 1:m) <= C*x[j]);

    #-------
    # SOLVE
    #-------
    set_optimizer_attribute(model, "TimeLimit", 30);
    optimize!(model)

    println();
    for j = 1:n
        allocated = false;
        for i = 1:n
            if (value(y[i,j]) >= 0.99999)
                print((i+1)," ");
                allocated = true;
            end
        end
        if allocated
            println();
        end
    end

end

solve_facility_location();



Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-20
Set parameter TimeLimit to value 30
Set parameter TimeLimit to value 30
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.6.0 22G436)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 60 rows, 930 columns and 1830 nonzeros
Model fingerprint: 0x08234484
Variable types: 0 continuous, 930 integer (930 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+00]
  Objective range  [3e-04, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 796.9481815
Presolve time: 0.00s
Presolved: 60 rows, 930 columns, 1830 nonzeros
Variable types: 0 continuous, 930 integer (930 binary)
Found heuristic solution: objective 628.7489657

Root relaxation: objective 8.167736e+01, 76 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     

After running the new service to customers for some time, Comma Computing notices that
customers would prefer to have a time window directly when booking a service visit. However,
making this possible would also require changing the allocation of customers to routes, as these
time windows now have to be considered in the VRP. Comma Computing ask you for help in
implementing this and provide a set of time windows for the previously considered instance in the
times sheet of the vrpData.xlsx file. Moreover, you can assume that traveling one kilometer
costs 2.5 minutes. As you believe that solving an exact formulation will be too time-consuming to
provide good solutions for this vehicle routing problem with time windows, you decide to use the
Clarke-Wright savings heuristic as a starting point.

sd

In [114]:
using DataFrames, XLSX, Dates

# Load the workbook
wb = XLSX.readxlsx("vrpData.xlsx")

# Get the specified sheets
sheet_cw = wb["cw"]
sheet_times = wb["times"]
sheet_repairTimes = wb["repairTimes"]
sheet_distance = wb["dist_min"]

# Read the data into an array and skip headers by adjusting range
data_cw = XLSX.getdata(sheet_cw, "A2:C436")
data_times = XLSX.getdata(sheet_times, "A2:C31")
data_repairTimes = XLSX.getdata(sheet_repairTimes, "A2:C32")
data_distance = XLSX.getdata(sheet_distance, "B2:AF32")  # Adjust the range to exclude headers

# Convert to DataFrame and assign column names
savings = DataFrame(data_cw, [:i, :j, :s_ij])
times = DataFrame(data_times, [:Customer, :Start, :End])
repair_times = DataFrame(data_repairTimes, [:Customer, :Service_Time_h, :Service_Time_m])
distance_matrix = DataFrame(data_distance, Symbol.(0:30))  # Assuming there are 30 customers

# Convert distance matrix strings to Float64 and multiply by 2.5 to get travel times in minutes
for col in names(distance_matrix)
    distance_matrix[!, col] = parse.(Float64, replace.(distance_matrix[!, col]))
end





MethodError: MethodError: no method matching promote_valuetype()
Closest candidates are:
  promote_valuetype(!Matched::Pair{K, V}) where {K, V} at set.jl:622
  promote_valuetype(!Matched::Pair{K, V}, !Matched::Pair...) where {K, V} at set.jl:623

In [103]:
using DataFrames, XLSX, Dates

# Load the workbook
wb = XLSX.readxlsx("vrpData.xlsx")

# Get the specified sheet
sheet_cw = wb["cw"]
sheet_times = wb["times"]
sheet_repairTimes = wb["repairTimes"]
sheet_distance = wb["distances"]

# Read the data into an array and skip headers by adjusting range
data_cw = XLSX.getdata(sheet_cw, "A2:C436")  # Adjust range if necessary
data_times = XLSX.getdata(sheet_times, "A2:C31")  # Adjust range if necessary
data_repairTimes = XLSX.getdata(sheet_repairTimes, "A2:B31")  # Adjust range if necessary
data_distance = XLSX.getdata(sheet_distance, "A1:AF32")  # Adjust the range according to your data size2
# Convert to DataFrame
savings = DataFrame(data_cw, [:i, :j, :s_ij])
times = DataFrame(data_times, [:Customer, :Start, :End])
repairTimes = DataFrame(data_repairTimes, [:Customer, :Service_Time])
distance_matrix = DataFrame(data_distance,:auto)
distance_matrix .= distance_matrix .* 2.5



# Function to convert time strings to Time type
function parse_time(str)
    h, m, s = split(str, ":")
    return Time(parse(Int, h), parse(Int, m), parse(Int, s))
end

# Apply time parsing if times are strings
if typeof(times[1, :Start]) == String
    times[!, :Start] = parse_time.(times[!, :Start])
    times[!, :End] = parse_time.(times[!, :End])
end

# Ensure the 's_ij' column is of type Float64
if savings[1, :s_ij] isa Number  # Check if the first element is a number
    savings[!, :s_ij] = float.(savings[!, :s_ij])  # Convert all to Float64 if not already
elseif savings[1, :s_ij] isa String
    savings[!, :s_ij] = parse.(Float64, savings[!, :s_ij])  # Parse strings to Float64
end













MethodError: MethodError: no method matching *(::String, ::Float64)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:591
  *(!Matched::T, ::T) where T<:Union{Float16, Float32, Float64} at float.jl:385
  *(::Union{AbstractChar, AbstractString}, !Matched::Union{AbstractChar, AbstractString}...) at strings/basic.jl:260
  ...

In [ ]:
using DataFrames, Dates

function check_time_feasibility(route, times, repair_times, travel_time_matrix, max_duration_minutes)
    total_time = 0
    # ... (the rest of your existing logic)

    for customer_idx in 2:length(route)
        prev_customer = route[customer_idx - 1]
        customer = route[customer_idx]
        travel_time = travel_time_matrix[prev_customer, customer]  # Use the travel time from the matrix
        service_time = repair_times.Time_m[repair_times.Customer .== customer][1]  # Use specific service time        

        # Calculate earliest possible start time for this customer
        possible_start = max(current_time + travel_time, customer_start)

        if possible_start > customer_end
            return false  # Cannot service this customer within their time window
        end

        # Update the current time after servicing this customer
        current_time = possible_start + service_time
        total_time += (travel_time.value + service_time.value)  # Use `.value` to get integer minutes

        if total_time > max_duration_minutes
            return false  # Exceeds the maximum allowed duration
        end
    end

    return true
end


# Initialize routes with each customer as a separate route
routes = [[i] for i in 1:nrow(times)]
#sort!(savings, :s_ij, rev=true)  # Sort savings in descending order
routes

# Implement the Clarke-Wright algorithm with time window constraints
for row in eachrow(savings)
    i, j = row.i, row.j
    route_i = findfirst(route -> i in route, routes)
    route_j = findfirst(route -> j in route, routes)

    if route_i != route_j
        merged_route = vcat(routes[route_i], routes[route_j])
        if check_time_feasibility(merged_route, times, 6 * 60)  # 6 hours converted to minutes
            routes[route_i] = merged_route
            deleteat!(routes, route_j)
        end
    end
end

# Display finalized routes
for (index, route) in enumerate(routes)
    println("Route $index: $route")
end


In [91]:
using DataFrames, Dates

# Function to check if the merged route respects the time windows and route duration limits
function check_time_feasibility(route, times, max_duration_minutes)
    total_time = 0
    current_time = Time(0, 0)  # Consider starting early in the morning

    for customer in route
        # Find the time window for this customer
        customer_start = times.Start[times.Customer .== customer][1]
        customer_end = times.End[times.Customer .== customer][1]
        
        # Assume fixed travel time to next customer (15 minutes for example)
        travel_time = Minute(15)
        service_time = Minute(60)  # assuming 1 hour service per customer
        

        # Calculate earliest possible start time for this customer
        possible_start = max(current_time + travel_time, customer_start)

        if possible_start > customer_end
            return false  # Cannot service this customer within their time window
        end

        # Update the current time after servicing this customer
        current_time = possible_start + service_time
        total_time += (travel_time.value + service_time.value)  # Use `.value` to get integer minutes

        if total_time > max_duration_minutes
            return false  # Exceeds the maximum allowed duration
        end
    end

    return true
end


# Initialize routes with each customer as a separate route
routes = [[i] for i in 1:nrow(times)]
#sort!(savings, :s_ij, rev=true)  # Sort savings in descending order
routes

# Implement the Clarke-Wright algorithm with time window constraints
for row in eachrow(savings)
    i, j = row.i, row.j
    route_i = findfirst(route -> i in route, routes)
    route_j = findfirst(route -> j in route, routes)

    if route_i != route_j
        merged_route = vcat(routes[route_i], routes[route_j])
        if check_time_feasibility(merged_route, times, 6 * 60)  # 6 hours converted to minutes
            routes[route_i] = merged_route
            deleteat!(routes, route_j)
        end
    end
end

# Display finalized routes
for (index, route) in enumerate(routes)
    println("Route $index: $route")
end


30-element Vector{Vector{Int64}}:
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]
 [10]
 ⋮
 [22]
 [23]
 [24]
 [25]
 [26]
 [27]
 [28]
 [29]
 [30]

In [92]:
using DataFrames, Dates

# Enhanced function to check route feasibility and print detailed timings
function check_time_feasibility(route, times, max_duration_minutes)
    total_time = 0
    current_time = Time(0, 0)  # Assume starting at midnight for simplification

    println("Checking route: $route")
    for customer in route
        customer_start = times.Start[times.Customer .== customer][1]
        customer_end = times.End[times.Customer .== customer][1]
        travel_time = Minute(15)  # Assuming 15 minutes travel time to each customer
        service_time = Minute(60)  # Assuming 1 hour service time

        possible_start = max(current_time + travel_time, customer_start)

        if possible_start > customer_end
            println("Cannot service customer $customer within their time window ($customer_start - $customer_end).")
            return false
        end

        actual_start = max(possible_start, customer_start)
        current_time = actual_start + service_time
        total_time += (travel_time.value + service_time.value)

        println("Customer $customer serviced from $actual_start to $current_time. Total route time so far: $total_time minutes.")

        if total_time > max_duration_minutes
            println("Exceeds the maximum allowed duration of $max_duration_minutes minutes.")
            return false
        end
    end

    return true
end

# Example data loading and processing (assumed to be correctly loaded and prepared)
# ... (Your data loading and initial processing code here)

# Assuming routes and savings are set up correctly
for row in eachrow(savings)
    i, j = row.i, row.j
    route_i = findfirst(route -> i in route, routes)
    route_j = findfirst(route -> j in route, routes)

    if route_i != route_j
        merged_route = vcat(routes[route_i], routes[route_j])
        println("\nAttempting to merge route containing customer $i with route containing customer $j.")
        if check_time_feasibility(merged_route, times, 6 * 60)  # 6 hours in minutes
            routes[route_i] = merged_route
            deleteat!(routes, route_j)
            println("Routes merged successfully.")
        else
            println("Failed to merge routes due to timing constraints.")
        end
    end
end

# Display finalized routes
println("\nFinal routes and timings:")
for (index, route) in enumerate(routes)
    println("Route $index: $route")
    if check_time_feasibility(route, times, 6 * 60)  # Re-check to print details
        println("Route $index is valid within time constraints.")
    else
        println("Route $index is NOT valid within time constraints.")
    end
end



Attempting to merge route containing customer 19 with route containing customer 20.
Checking route: [19, 20]
Customer 19 serviced from 10:00:00 to 11:00:00. Total route time so far: 75 minutes.
Customer 20 serviced from 11:15:00 to 12:15:00. Total route time so far: 150 minutes.
Routes merged successfully.

Attempting to merge route containing customer 19 with route containing customer 24.
Checking route: [19, 20, 24]
Customer 19 serviced from 10:00:00 to 11:00:00. Total route time so far: 75 minutes.
Customer 20 serviced from 11:15:00 to 12:15:00. Total route time so far: 150 minutes.
Customer 24 serviced from 12:30:00 to 13:30:00. Total route time so far: 225 minutes.
Routes merged successfully.

Attempting to merge route containing customer 23 with route containing customer 26.
Checking route: [23, 26]
Customer 23 serviced from 15:00:00 to 16:00:00. Total route time so far: 75 minutes.
Customer 26 serviced from 16:15:00 to 17:15:00. Total route time so far: 150 minutes.
Routes merg